In [140]:
# Code for data collection
from bs4 import BeautifulSoup as bs
import requests as rq
import re

langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}

page = rq.get("https://www.linkedin.com/jobs/search?keywords=Software%2Bengineer&location=United%20States&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")
soup = bs(page.content, 'html.parser')
items = soup.find_all(class_="base-card__full-link")
print(langs.values())
salary_match = "\$?\d{0,3},?\d{1,3},\d{3,3}\.?\d{0,2}"
item = items[0]
for item in items:
    url = item.get('href')
    print(url + "\n")
    posting_page = rq.get(url)
    s = bs(posting_page.content, 'html.parser')
    text = s.get_text()
    sal_matches = list(map(lambda x: float(re.sub("[^0-9\.]", "", x)), re.findall(salary_match, text)))
    for k, v in langs.items(): 
        if re.search(k, text, flags=re.IGNORECASE) != None:
            langs[k].extend(sal_matches)
    
print(langs)

dict_values([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])
https://www.linkedin.com/jobs/view/c%2B%2B-software-engineer-at-jobot-3034297216?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=FxKZ5GltoHktaVtQRXH%2ByA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/software-engineer-c%23-at-itw-3039563164?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=xtJXCmxyuGtI7%2BKZDHbmsw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/%232833-software-engineer-react-native-at-it-motives-3029776271?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=t1z6oqJDI65vv03pGROiTA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/backend-software-engineer-data-ingestion-at-gridium-3042407691?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=lJp5KTl7jLJaBKFVYi5DOw%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-ca

In [197]:
# import csv
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller
import re



langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}
keyboard = Controller()
scraped_jobs = []
job_set = set()

# Initialize Driver
url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    
    
    

    keyboard.press(Key.esc)
    keyboard.release(Key.esc)
    
    num_pages_to_search = 2
    num_pages_searched = 0
    num_jobs_searched = 0
    
    # extract the job data in "num_pages_to_search" pages
    while num_pages_searched < num_pages_to_search:
        # Each job
        jobs_on_page = driver.find_elements_by_class_name('react-job-listing')
        
        for job_card in jobs_on_page:
            # sleep to load the info 
            sleep(1)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            
            try:
                text = driver.find_element_by_class_name('jobDescriptionContent').text.lower()
                salary = process_salary(driver.find_element_by_class_name('css-y2jiyn').text)
                company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
                job_position = driver.find_element_by_class_name('css-1j389vi').text
                format_data((company, job_position, salary))
            except NoSuchElementException:
                print("No Salary")

            job_card.click()

        sleep(1)
        try:
            text = driver.find_element_by_class_name('jobDescriptionContent').text.lower()
            salary = process_salary(driver.find_element_by_class_name('css-y2jiyn').text)
            company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
            job_position = driver.find_element_by_class_name('css-1j389vi').text
            format_data((company, job_position, salary))
        except NoSuchElementException:
            print("No Salary")
        num_pages_to_search += 1
        print(langs)
        
        # go to next page
        driver.find_element_by_class_name('nextButton').click()
        sleep(2)
    record_data(scraped_jobs)

# Used to get the name of Job company without star ratings
def process_company(company):
    try:
        if driver.find_element_by_class_name('css-1m5m32b') != None:
            return company[0:-4]
    except NoSuchElementException:
        return company

# $70,000 /yr (est.) => 70000
# $35 /hr (est.) => 35*52*40
def process_salary(salary):
    new_salary = salary.split(' ', 1)
    if new_salary[1] == '/yr (est.)':
        return new_salary[0][1:]
    else:
        return str(float(new_salary[0][1:]) * 52 * 40)
        
# Format the data received from parsing a Job Card
def format_data(job_row):
    if job_row not in job_set:
        print(job_row)
        scraped_jobs.append(job_row)
        job_set.add(job_row)

# At the end, I want to record all my unique jobs to a CSV file
def record_data(records):
    with open('salaries.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Company','Job Title', 'Salary'])
        writer.writerows(records)
        
# Run my main program
main()




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/eddiexu/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache


('Oracle', 'Software Developer 5 2022P', '145,517')
('KUBRA', 'Java Software Engineer', '112,665')
('KOHLS', 'Software Engineer', '102,206')
No Salary
('BOEING', 'Entry-Level Software Engineer for Embedded Training', '92,926')
('Renaissance', 'Software Engineer III', '106,500')
('KOHLS', 'Platform Engineer - Middleware - REMOTE Opportunity', '83,681')
('Simons Foundation', 'Software Engineer, SFARI Base (Django)', '90,451')
('jobfuture.ai', 'HYBRID Fullstack Software Engineer (UP to 210k Plus Equity) React + Typescript and Django', '162,500')
('Plaudit Design', 'Full-Stack Programmer (Remote or St Paul Office)', '54,254')
('HP', 'Entry Level Full Stack Software Engineer', '106,018')
No Salary
('Loop', 'Software Engineer, Fullstack', '135,000')
('Liberty Mutual Insurance', 'Software Engineer - Software Delivery (Remote)', '98,025')
('Confidential', 'Senior Software Engineer', '110,000')
('Indeed', 'Software Engineer I - Enterprise', '127,000')
('MIM Software', 'Software Services Enginee

In [172]:
import csv
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url
def save_data_to_file(records):
    """Save data to csv file"""
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'JobUrl'])
        writer.writerows(records)
def get_record(card):
    """Extract job data from single card"""
    job_title = card.find_element_by_class_name('jobtitle').text
    company = card.find_element_by_class_name('company').text
    location = card.find_element_by_class_name('location').text
    post_date = card.find_element_by_class_name('date').text
    extract_date = datetime.today().strftime('%Y-%m-%d')
    summary = card.find_element_by_class_name('summary').text
    job_url = card.find_element_by_class_name('jobtitle').get_attribute('href')
    return (job_title, company, location, post_date, extract_date, summary, job_url)
def get_page_records(cards, job_list, url_set):
    """Extract all cards from the page"""
    for card in cards:
        record = get_record(card)
        # add if job title exists and not duplicate
        if record[0] and record[-1] not in url_set:
            job_list.append(record)
            url_set.add(record[-1])
def main(position, location):
    """Run the main program routine"""
    scraped_jobs = []
    scraped_urls = set()
    
    url = get_url(position, location)
    
    # setup web driver
    driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
    driver.get(url)
    # extract the job data
    while True:
        cards = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
        get_page_records(cards, scraped_jobs, scraped_urls)
        try:
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            driver.find_element_by_id('popover-x').click()  # to handle job notification popup
            get_page_records(cards, scraped_jobs, scraped_urls)
            continue
    
    # close driver and save records
    driver.quit()
    save_data_to_file(scraped_jobs)
# run a search
main('python developer', 'charlotte nc')

In [148]:
import csv
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller
import re
url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)     



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/eddiexu/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache
